In [105]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [106]:
# defining the model

model = pyo.ConcreteModel()

# decision variables
model.x1 = pyo.Var(within = pyo.NonNegativeReals)
model.x2 = pyo.Var(within = pyo.NonNegativeReals)

x1 = model.x1
x2 = model.x2

# objective function
model.obj = pyo.Objective(expr = 4*x1 + 3*x2, sense = pyo.maximize)

#constraints
model.const1 = pyo.Constraint(expr = x1 + x2 <= 40)
model.const2 = pyo.Constraint(expr = 2*x1 + x2 <= 60)

optm = SolverFactory('glpk')
results = optm.solve(model)
print(results)
print('objective function= ', model.obj())
print('x1 =', model.x1())
print('x2 =', model.x2())



Problem: 
- Name: unknown
  Lower bound: 140.0
  Upper bound: 140.0
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 2
  Number of nonzeros: 4
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.05196094512939453
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

objective function=  140.0
x1 = 20.0
x2 = 20.0


In [107]:
model = pyo.ConcreteModel()

# Sets 
model.i = pyo.Set(initialize = ['Desk', 'Table', 'Chair'])

#parameters
model.L = pyo.Param(model.i, initialize = {'Desk': 8, 'Table': 6, 'Chair':1})
L = model.L

model.F = pyo.Param(model.i, initialize = {'Desk': 4, 'Table': 2, 'Chair':1.5})
F = model.F

model.C = pyo.Param(model.i, initialize = {'Desk': 2, 'Table': 1.5, 'Chair':0.5})
C = model.C

model.P = pyo.Param(model.i, initialize = {'Desk': 60, 'Table': 30, 'Chair':20})
P = model.P


In [108]:
#Decision variables
model.x = pyo.Var(model.i, within = pyo.NonNegativeReals)
x = model.x

In [109]:
#Objective function
def Objective_rule(model):
    return  sum(P[i]*x[i] for i in model.i)

In [110]:
model.Obj = pyo.Objective(rule = Objective_rule, sense = pyo.maximize)

In [111]:
#Constraints
def Constraint1 (model, i):
    return sum(L[i] * x[i] for i in model.i) <= 48

def Constraint2 (model, i):
    return sum(F[i] * x[i] for i in model.i) <= 20

def Constraint3 (model, i):
    return sum(C[i] * x[i] for i in model.i) <= 8

def Constraint4 (model, i):
    return x['Table'] <= 5

In [112]:
model.Const1 = pyo.Constraint(model.i, rule = Constraint1)
model.Const2 = pyo.Constraint(model.i, rule = Constraint2)
model.Const3 = pyo.Constraint(model.i, rule = Constraint3)
model.Const4 = pyo.Constraint(model.i, rule = Constraint4)

In [113]:
#Solve
Solver = SolverFactory('glpk')
results = Solver.solve(model)

print(results)
print('Objective Function = ', model.Obj())

for i in model.i:
    print('Number of ', i, 'produced = ', x[i]())


Problem: 
- Name: unknown
  Lower bound: 280.0
  Upper bound: 280.0
  Number of objectives: 1
  Number of constraints: 12
  Number of variables: 3
  Number of nonzeros: 30
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.08508849143981934
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Objective Function =  280.0
Number of  Desk produced =  2.0
Number of  Table produced =  0.0
Number of  Chair produced =  8.0


In [114]:
model = pyo.ConcreteModel()

In [115]:
#Sets

model.i = pyo.RangeSet(1,3)
model.j = pyo.RangeSet(1,4)

# model.i = pyo.Set('plant1','plant2','plant3')
# model.j = pyo.Set('city1','city2','city3','city4')


In [116]:
# Parameters
model.S = pyo.Param(model.i, initialize = {1:35, 2:50, 3:40})
S = model.S

model.D = pyo.Param(model.j, initialize = {1:45, 2:20, 3:30, 4:30})
D = model.D

model.P = pyo.Param(model.i, model.j, initialize = {(1,1):8, (1,2):6, (1,3):10, (1,4):9, (2,1):9, (2,2):12,(2,3):13,(2,4):7,(3,1):14,(3,2):9,(3,3):16, (3,4):5})
P = model.P

In [117]:
#Variables
model.x = pyo.Var(model.i, model.j, within = pyo.NonNegativeReals)
x = model.x

In [118]:
#Objective Function
def Objective_rule(model):
    return sum(sum(P[i,j]*x[i,j] for i in model.i) for j in model.j)

model.Objf = pyo.Objective(rule = Objective_rule, sense = pyo.minimize)

In [119]:
#Constraints
def Constraint1(model,i):
    return sum(x[1,j] for j in model.j) <= S[1]

def Constraint2(model,i):
    return sum(x[2,j] for j in model.j) <= S[1]

def Constraint3(model,i):
    return sum(x[3,j] for j in model.j) <= S[1]

def Constraint4(model,j):
    return sum(x[i,1] for i in model.i) >= D[1]

def Constraint5(model,j):
    return sum(x[i,2] for i in model.i) >= D[2]

def Constraint6(model,j):
    return sum(x[i,3] for i in model.i) >= D[3]

def Constraint7(model,j):
    return sum(x[i,4] for i in model.i) >= D[4]

In [120]:
model.Const1 = pyo.Constraint(model.j, rule = Constraint1)
model.Const2 = pyo.Constraint(model.j, rule = Constraint2)
model.Const3 = pyo.Constraint(model.j, rule = Constraint3)
model.Const4 = pyo.Constraint(model.i, rule = Constraint4)
model.Const5 = pyo.Constraint(model.i, rule = Constraint5)
model.Const6 = pyo.Constraint(model.i, rule = Constraint6)
model.Const7 = pyo.Constraint(model.i, rule = Constraint7)

In [124]:
Solver = SolverFactory('glpk')
results = Solver.solve(model)

print(results)
#print('Objective function = ', model.Objf())

#for i in model.i:
   # for j in model.j:
        #print("Electricity send from plant ", i, 'to city ', j, '= ', x[i][j]())


Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 24
  Number of variables: 12
  Number of nonzeros: 84
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: other
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.08617234230041504

